In [1]:
import pandas as pd
import sqlite3 as sql

**Імпорт усіх таблиць з реляційної БД**

In [2]:
brands = pd.read_csv('/kaggle/input/bike-store-sample-database/brands.csv')
categories = pd.read_csv('/kaggle/input/bike-store-sample-database/categories.csv')
customers = pd.read_csv('/kaggle/input/bike-store-sample-database/customers.csv')
order_items = pd.read_csv('/kaggle/input/bike-store-sample-database/order_items.csv')
orders = pd.read_csv('/kaggle/input/bike-store-sample-database/orders.csv')
products = pd.read_csv('/kaggle/input/bike-store-sample-database/products.csv')
staffs = pd.read_csv('/kaggle/input/bike-store-sample-database/staffs.csv')
stocks = pd.read_csv('/kaggle/input/bike-store-sample-database/stocks.csv')
stores = pd.read_csv('/kaggle/input/bike-store-sample-database/stores.csv')

**Створення зв'язку з БД та вставлення даних**

In [5]:
con = sql.connect('analysis.db')

brands.to_sql('brands', con, index=False, if_exists='replace')
categories.to_sql('categories', con, index=False, if_exists='replace')
customers.to_sql('customers', con, index=False, if_exists='replace')
order_items.to_sql('order_items', con, index=False, if_exists='replace')
orders.to_sql('orders', con, index=False, if_exists='replace')
products.to_sql('products', con, index=False, if_exists='replace')
staffs.to_sql('staffs', con, index=False, if_exists='replace')
stocks.to_sql('stocks', con, index=False, if_exists='replace')
stores.to_sql('stores', con, index=False, if_exists='replace')

3

**1. Розглянемо кількість виконаних замовлень (order status 4/completed), за магазинами**

In [7]:
CountDoneByStore = """
SELECT store_id, COUNT(*)
FROM orders
WHERE order_status = 4
GROUP BY store_id
ORDER BY COUNT(*) DESC
"""

pd.read_sql_query(CountDoneByStore, con)

,store_id,COUNT(*)
0,2,1019
1,1,284
2,3,142


**2. За допомогою CTE, розглянемо відсоток виконаних замовлень**

In [12]:
PercentageDoneByStore = """
WITH TotalOrders AS (
    SELECT store_id, COUNT(*) AS total_orders
    FROM orders
    GROUP BY store_id
),
CompletedOrders AS (
    SELECT store_id, COUNT(*) AS completed_orders
    FROM orders
    WHERE order_status = 4
    GROUP BY store_id
)
SELECT c.store_id, c.completed_orders, t.total_orders,
    ROUND(c.completed_orders * 100.0 / t.total_orders, 2) as percentage_completed 
FROM CompletedOrders c
JOIN TotalOrders t ON c.store_id = t.store_id
ORDER BY c.completed_orders DESC
"""

pd.read_sql_query(PercentageDoneByStore, con)

,store_id,completed_orders,total_orders,percentage_completed
0,2,1019,1093,93.23
1,1,284,348,81.61
2,3,142,174,81.61


3. Знайдемо загальну кількість клієнтів, що здійснили замовлення

In [15]:
CountClients = """
SELECT COUNT(DISTINCT customer_id) AS total_customers_with_orders
FROM orders
"""

pd.read_sql_query(CountClients, con)

,total_customers_with_orders
0,1445


**4. За допомогою підзапиту знайдемо клієнтів, загальна сума покупок яких є вищою від середньої (Наприклад для пропонування знижки)**

In [14]:
BiggerThenAvgAmount = """
SELECT c.customer_id, CONCAT(c.first_name, ' ', c.last_name) AS customer_name,
    SUM(oi.list_price * oi.quantity * (1 - oi.discount)) AS total_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY c.customer_id, customer_name
HAVING SUM(oi.list_price * oi.quantity * (1 - oi.discount)) > (
    SELECT AVG(sub.total_amount)
    FROM (
        SELECT c.customer_id,
        SUM(oi.list_price * oi.quantity * (1 - oi.discount)) AS total_amount
        FROM customers c
        JOIN orders o ON c.customer_id = o.customer_id
        JOIN order_items oi ON o.order_id = oi.order_id
        GROUP BY c.customer_id
    ) AS sub
)
"""

pd.read_sql_query(BiggerThenAvgAmount, con)

,customer_id,customer_name,total_spent
0,1,Debra Burks,27888.1834
1,2,Kasha Todd,19329.0849
2,3,Tameka Fisher,24051.5279
3,4,Daryl Spence,21150.8927
4,5,Charolette Rice,17520.2919
...,...,...,...
537,1440,Ernest Rollins,10360.2550
538,1441,Jamaal Morrison,9173.3817
539,1442,Cassie Cline,6987.2487
540,1443,Lezlie Lamb,10372.0934


**5. Створимо "широку" таблицю, включивши в неї необхідні колонки з БД для подальшого аналізу в Excel**

In [16]:
BigTableQuery = """
SELECT o.order_id, o.customer_id, o.order_status, o.order_date, o.shipped_date, o.store_id,
    CONCAT(c.first_name, ' ', c.last_name) AS customer_name, c.city as customer_city, c.state as customer_state,
    s.store_name, s.city as store_city,
    oi.product_id, oi.quantity, oi.list_price, oi.discount, (oi.list_price * oi.quantity * (1 - oi.discount)) as total_spent,
    p.product_name, p.brand_id, p.category_id, 
    cat.category_name,
    b.brand_name
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN stores s ON o.store_id = s.store_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
JOIN categories cat ON p.category_id = cat.category_id
JOIN brands b ON p.brand_id = b.brand_id
"""

pd.read_sql_query(BigTableQuery, con)

,order_id,customer_id,order_status,order_date,shipped_date,store_id,customer_name,customer_city,customer_state,store_name,...,product_id,quantity,list_price,discount,total_spent,product_name,brand_id,category_id,category_name,brand_name
0,1,259,4,2016-01-01,2016-01-03,1,Johnathan Velazquez,Pleasanton,CA,Santa Cruz Bikes,...,4,1,2899.99,0.20,2319.9920,Trek Fuel EX 8 29 - 2016,9,6,Mountain Bikes,Trek
1,1,259,4,2016-01-01,2016-01-03,1,Johnathan Velazquez,Pleasanton,CA,Santa Cruz Bikes,...,8,2,1799.99,0.07,3347.9814,Trek Remedy 29 Carbon Frameset - 2016,9,6,Mountain Bikes,Trek
2,1,259,4,2016-01-01,2016-01-03,1,Johnathan Velazquez,Pleasanton,CA,Santa Cruz Bikes,...,10,2,1549.00,0.05,2943.1000,Surly Straggler - 2016,8,4,Cyclocross Bicycles,Surly
3,1,259,4,2016-01-01,2016-01-03,1,Johnathan Velazquez,Pleasanton,CA,Santa Cruz Bikes,...,16,2,599.99,0.05,1139.9810,Electra Townie Original 7D EQ - 2016,1,3,Cruisers Bicycles,Electra
4,1,259,4,2016-01-01,2016-01-03,1,Johnathan Velazquez,Pleasanton,CA,Santa Cruz Bikes,...,20,1,599.99,0.20,479.9920,Electra Townie Original 7D EQ - Women's - 2016,1,3,Cruisers Bicycles,Electra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4717,1614,135,3,2018-11-28,None,3,Dorthey Jackson,Houston,TX,Rowlett Bikes,...,159,2,2299.99,0.07,4277.9814,Trek Emonda ALR 6 - 2018,9,7,Road Bikes,Trek
4718,1614,135,3,2018-11-28,None,3,Dorthey Jackson,Houston,TX,Rowlett Bikes,...,213,2,269.99,0.20,431.9840,Electra Cruiser 1 - 2016/2017/2018,1,3,Cruisers Bicycles,Electra
4719,1615,136,3,2018-12-28,None,3,Sarita Parks,Forney,TX,Rowlett Bikes,...,182,1,2499.99,0.20,1999.9920,Trek Domane SL 5 Disc - 2018,9,7,Road Bikes,Trek
4720,1615,136,3,2018-12-28,None,3,Sarita Parks,Forney,TX,Rowlett Bikes,...,197,2,2299.99,0.20,3679.9840,Trek Verve+ Lowstep - 2018,9,5,Electric Bikes,Trek


**6. Зберігаємо дані, як csv для подальшого аналізу в Excel**

In [17]:
result_df = pd.read_sql_query(BigTableQuery, con)
result_df.to_csv('/kaggle/working/result.csv', index=False)